# Fetch Pathways/Reactions/Catalysis and Reactants from Reactome

In [ ]:
import pickle
import requests
from collections import defaultdict

import pandas as pd
import numpy as np
import copy
import json
import rdflib
from rdflib import XSD

from bluegraph import PandasPGFrame
from bluegraph.backends.neo4j import pgframe_to_neo4j

from bmo_tools.kbs.reactome import *
from bmo_tools.neo4j import (ontology_to_neo4j, clean_db,
                             label_top_level, label_entities, execute)
from bmo_tools.ontologies import (normalize_uris, subontology_from_term)
from bmo_tools.kbs.gene_kbs import get_gene_data, get_orthologues

Set pathways of interest

In [ ]:
pathways = {
    "Glycolysis": ["R-MMU-70171", "R-HSA-70171", "R-RNO-70171"],
    "Glycogenolysis": ["R-HSA-70221", "R-MMU-70221", "R-RNO-70221"],
    "Pentose phosphate pathway": ["R-HSA-71336", "R-MMU-71336", "R-RNO-71336"],
    "Citric acid cycle (TCA cycle)": ["R-HSA-71403", "R-MMU-71403", "R-RNO-71403"]
}

## Get pathway records

In [ ]:
pathway_data = {}
edges = defaultdict(set)
for k, v in pathways.items():
    for vv in v:
        vv_pathways, vv_edges = get_pathway_data(vv)
        pathway_data.update(vv_pathways)
        edges.update(vv_edges)

Convert to a dataframe

In [ ]:
pathway_df = pd.DataFrame(pathway_data.values())
pathway_df = clean_reactome_df(pathway_df)
pathway_df["@id"] = pathway_df["reactome_id"]
pathway_df.head(3)

## Get individual reactions and reactants

In [ ]:
# Get reactions
reaction_data = {}
reactant_data = {}
new_edges = {}
for (s, t), edge_types in edges.items():
    if "isPartOf" in edge_types and s not in pathway_data:
        props, s_edges, d = get_reaction_data(s, reactant_data)
        reaction_data[s] = props
        new_edges.update(s_edges)

In [ ]:
edges.update(new_edges)

Convert to a dataframe

In [ ]:
reaction_df = pd.DataFrame(reaction_data.values())
reaction_df = clean_reactome_df(reaction_df)
reaction_df["@id"] = reaction_df["reactome_id"]

In [ ]:
reaction_df.shape

Add 'positivelyRegulates' and 'negativelyRegulates' edges.

In [ ]:
for r_id, regulation_data in reaction_df[["@id", "regulatedBy"]].values:
    if not isinstance(regulation_data, float):
        for el in regulation_data:
            regulator, edge_types = get_regulation_relations(
                el["stId"] if "stId" in el else el["dbId"], reactant_data)
            edges[(regulator, r_id)] = {edge_types}

Add 'reverseReaction' edges

In [ ]:
if "reverseReaction" in reaction_df.columns:
    for r_id, reverse_reaction in reaction_df[["@id", "reverseReaction"]].values:
        if not isinstance(reverse_reaction, float):
            if reverse_reaction["stId"] not in reaction_data:
                props, new_edges, new_edge_props = get_reaction_data(
                    reverse_reaction["stId"], reactant_data)
                reaction_data[reverse_reaction["stId"]] = props
                edges.update(new_edges)

In [ ]:
remove = [
    "catalystActivity", "regulatedBy",
    "catalystActivityReference",
    "regulationReference", "reverseReaction"
]
for c in remove:
    if c in reaction_df.columns:
        reaction_df = reaction_df.drop(columns=[c])

Process reactant data

In [ ]:
reactant_df = pd.DataFrame([
    {"stId": k , **v}
    for k, v in reactant_data.items()
])
reactant_df = clean_reactome_df(reactant_df)
reactant_df["@id"] = reactant_df["reactome_id"]

## Get enzymes

In [ ]:
catalysts = {}
new_edges = {}
edges_to_remove = set()
for (s, t), types in edges.items():
    if "catalyzedBy" in types:
        tt_edges, tt_remove_edges = get_protein_data(t, s, catalysts)
        new_edges.update(tt_edges)
        edges_to_remove.update(tt_remove_edges)

In [ ]:
edges.update(new_edges)

In [ ]:
for e in edges_to_remove:
    if e in edges:
        del edges[e]

In [ ]:
catalysts_df = pd.DataFrame(catalysts.values())
catalysts_df = clean_reactome_df(catalysts_df)
catalysts_df = catalysts_df.drop_duplicates(subset="reactome_id")
catalysts_df["@id"] = catalysts_df["reactome_id"]
if "goCellularComponent" in catalysts_df.columns:
    catalysts_df["goCellularComponent"] = catalysts_df["goCellularComponent"].apply(
        lambda x: f"{x['databaseName']}:{x['accession']}" if not isinstance(x, float) else np.nan)
catalysts_df["hasModifiedResidue"] = catalysts_df["hasModifiedResidue"].apply(
    lambda x: [el["displayName"] for el in x] if not isinstance(x, float) else np.nan)

Process edges

In [ ]:
edge_df = pd.DataFrame(
    [(s, t, types) for (s, t), types in edges.items()],
    columns=["@source_id", "@target_id", "@type"]
)
edge_df = edge_df.set_index(["@source_id", "@target_id"])

Save the data

In [ ]:
pathway_df.to_pickle("../data/reactome/pathways.pkl")
reaction_df.to_pickle("../data/reactome/reactions.pkl")
reactant_df.to_pickle("../data/reactome/reactants.pkl")
catalysts_df.to_pickle("../data/reactome/catalysts.pkl")
edge_df.to_pickle("../data/reactome/edges.pkl")

In [ ]:
# pathway_df = pd.read_pickle("../data/reactome/pathways.pkl")
# reaction_df = pd.read_pickle("../data/reactome/reactions.pkl")
# reactant_df = pd.read_pickle("../data/reactome/reactants.pkl")
# catalysts_df = pd.read_pickle(../data/reactome/catalysts.pkl")
# edge_df = pd.read_pickle("../data/reactome/edges.pkl")

## Load Molecular Systems Ontology

In [ ]:
normalize_uris("../../ontologies/bmo.ttl", "https://neuroshapes.org/" , "../../ontologies/bmo.ttl")
normalize_uris("../../ontologies/molecular_systems_ontology.ttl", "https://neuroshapes.org/", "../../ontologies/molecular_systems_ontology.ttl")

In [ ]:
g = rdflib.Graph()
g.parse("../../ontologies/bmo.ttl", format="turtle")
g.parse("../../ontologies/molecular_systems_ontology.ttl", format="turtle")

## Create a PGFrame

In [ ]:
frame = PandasPGFrame.from_ontology(
    rdf_graph=g, remove_prop_uris=True)

In [ ]:
frame._nodes["@type"] = [set()] * frame.number_of_nodes()

In [ ]:
reactome_nodes = pd.concat([pathway_df, reaction_df, reactant_df, catalysts_df])
reactome_nodes = reactome_nodes.drop_duplicates(subset="@id")
reactome_nodes = reactome_nodes.set_index("@id")

In [ ]:
frame._nodes = pd.concat([frame._nodes, reactome_nodes])
frame._nodes.loc[frame._nodes["chebi_id"].apply(lambda x: x is None), "chebi_id"] = np.nan

In [ ]:
frame._edges =  pd.concat([frame._edges, edge_df])

In [ ]:
new_edges = [
    (el, "Protein")
    for el in frame._nodes[frame._nodes["@type"].apply(lambda x: "PROTEIN" in x)].index
] + [
    (el, "Metabolite")
    for el in frame._nodes[frame._nodes["@type"].apply(lambda x: "METABOLITE" in x)].index
] + [
    (el, "Complex")
    for el in frame._nodes[frame._nodes["@type"].apply(lambda x: "COMPLEX" in x)].index
] + [
    (el, "Biochemical Reaction")
    for el in frame._nodes[frame._nodes["@type"].apply(lambda x: "BIOCHEMICAL_REACTION" in x)].index
] + [
    (el, "Metabolic Pathway")
    for el in frame._nodes[frame._nodes["@type"].apply(lambda x: "PATHWAY" in x)].index
]

In [ ]:
frame.add_edges(new_edges)
for s, t in new_edges:
    frame._edges.loc[(s, t), "@type"] = {"IS_INSTANCE_OF"}

## Get gene data and link it to respective proteins

In [ ]:
gene_mapping = frame._nodes[
    frame._nodes["@type"].apply(lambda x: "PROTEIN" in x)][["gene", "species", "synonyms"]].to_dict("index")

In [ ]:
gene_records = {}
for protein_id, data in gene_mapping.items():
    if data["gene"] not in gene_records and not isinstance(data["species"], float):
        record = get_gene_data(data["gene"], data["species"].replace(" ", "_").lower())
        if len(record) == 0:
            record = get_gene_data(data["synonyms"][0], data["species"].replace(" ", "_").lower())
        record["species"] = data["species"]
        if 'prefLabel' in record:
            record["label"] = f"{record['prefLabel']} ({record['species']})"
        gene_records[data["gene"]] = record

In [ ]:
gene_df = pd.DataFrame(gene_records.values())
gene_df["@id"] = gene_df["uniprot_ac"]
gene_df["@type"] = "GENE"
gene_df = gene_df.dropna()

In [ ]:
gene_df.to_pickle("../data/reactome/genes.pkl")

In [ ]:
go_edges = gene_df[["@id", "go_edges"]]

In [ ]:
gene_df = gene_df.drop(columns=["go_edges"]).drop_duplicates("@id")
gene_df["xrefs"] = gene_df["xrefs"].apply(lambda x: {f"{k}:{v}" for k, v in x.items()})
frame.add_nodes(gene_df["@id"])
for c in gene_df.columns:
    if c != "@id":
        frame.add_node_properties(gene_df[["@id", c]])

In [ ]:
instance_edges = set([
    (el, "Gene")   
    for el in gene_df["@id"] 
])
frame.add_edges(instance_edges)
for s, t in instance_edges:
    frame._edges.loc[(s, t), "@type"] = {"IS_INSTANCE_OF"}

In [ ]:
translates_edges = set()
for k, v in gene_mapping.items():
    if v["gene"] in gene_records:
        if "uniprot_ac" in gene_records[v["gene"]]:
            translates_edges.add((gene_records[v["gene"]]["uniprot_ac"], k))
        else:
            translates_edges.add((v["gene"], k))

In [ ]:
frame.add_edges(translates_edges)
for s, t in translates_edges:
    frame._edges.loc[(s, t), "@type"] = {"translatesInto"}
frame._nodes.loc[frame._nodes["@type"].isna(), "@type"] = frame._nodes.loc[
    frame._nodes["@type"].isna(), "@type"].apply(lambda x: set())

## Get orthologous genes

In [ ]:
species_of_interest = set(
    [
        el.lower().replace(" ", "_") for el in frame._nodes["species"].unique()
        if not isinstance(el, float)
    ]
)

In [ ]:
ensembl_genes = {v["ensembl_id"]: k for k, v in gene_records.items() if "ensembl_id" in v}

In [ ]:
# Fetch orthologous genes from Ensembl
visited = set()
orthologues = []
for gene in ensembl_genes:
    if gene not in visited:
        result = get_orthologues(gene, species_of_interest)
        group = set([gene] + list(result.values()))
        for el in group:
            visited.add(el)
        orthologues.append(group)

In [ ]:
with open("../data/reactome/orthologous_genes.pkl", "wb") as f:
    pickle.dump(orthologues, f)

In [ ]:
new_edges = set()
for group in orthologues:
    nodes_to_connect = []
    for el in group:
        node_ids = frame._nodes[frame._nodes["ensembl_id"] == el].index.tolist()
        if len(node_ids) == 1:
            nodes_to_connect.append(node_ids[0])
    if len(nodes_to_connect) > 1:
        for el in nodes_to_connect[1:]:
            new_edges.add((nodes_to_connect[0], el))

In [ ]:
frame.add_edges(new_edges)
for s, t in new_edges:
    frame._edges.loc[(s, t), "@type"] = {"hasOrthologue"}

## Merge gene data with Gene ontology

In [ ]:
go = rdflib.Graph()
go.parse("../../ontologies/go.ttl", format="turtle")

Collect a subset of GO terms to include

In [ ]:
go_terms_to_include = {}

# Collect terms referenced by genes
for g, record in gene_records.items():
    if "go_edges" in record:
        for _, t in record["go_edges"]:
            if t not in go_terms_to_include:
                for s in go.subjects(
                        rdflib.URIRef("http://www.geneontology.org/formats/oboInOwl#id"),
                        rdflib.Literal(t, datatype=XSD.string)):
                    go_terms_to_include[t] = s
                    break

In [ ]:
# Collect terms referenced by pathways/reactions/reactants/enzymes
located_in_edges = [
    (el["@id"], el["compartment"])
    for el in (
        pathway_df.loc[pathway_df["compartment"].notna(), ["@id", "compartment"]].to_dict("records") +
        catalysts_df.loc[catalysts_df["compartment"].notna(), ["@id", "compartment"]].to_dict("records") +
        reaction_df[["@id", "compartment"]].to_dict("records") +
        reactant_df.loc[reactant_df["compartment"].notna(), ["@id", "compartment"]].to_dict("records")
    )
]
involved_in_edges = [
    (el["@id"], [el["goBiologicalProcess"]])
    for el in pathway_df[["@id", "goBiologicalProcess"]].to_dict("records")
]

new_go_terms = set(
    sum([el for _, el in located_in_edges + involved_in_edges], [])
)

In [ ]:
for t in new_go_terms:
    if t not in go_terms_to_include:
        for s in go.subjects(rdflib.URIRef("http://www.geneontology.org/formats/oboInOwl#id"), rdflib.Literal(t, datatype=XSD.string)):
            go_terms_to_include[t] = s
            break

In [ ]:
selected_go = rdflib.Graph()
for t in go_terms_to_include.values():
    subontology = subontology_from_term(go, t, top_down=False, closed=True)
    selected_go += subontology

In [ ]:
go_frame = PandasPGFrame.from_ontology(rdf_graph=selected_go, remove_prop_uris=True)
go_frame.remove_isolated_nodes()
go_frame.rename_node_properties({"id": "go_id"})

In [ ]:
frame._nodes = pd.concat([frame._nodes, go_frame._nodes])
frame._edges = pd.concat([frame._edges, go_frame._edges])

In [ ]:
new_go_edges = {}
for row in go_edges.to_dict("records"):
    source  = row["@id"]
    for rel, go_term in row["go_edges"]:
        target = frame._nodes[frame._nodes["go_id"] == go_term].index[0]
        if (source, target) in new_go_edges:
            new_go_edges[(source, target)].add(rel.replace(" ", "_"))
        else:
            new_go_edges[(source, target)] = {rel.replace(" ", "_")}

In [ ]:
for source, go_terms in located_in_edges:
    for term in go_terms:
        target = frame._nodes[frame._nodes["go_id"] == term].index[0]
        if (source, target) in new_go_edges:
            new_go_edges[(source, target)].add("located_in")
        else:
            new_go_edges[(source, target)] = {"located_in"}


for source, go_terms in involved_in_edges:
    for term in go_terms:
        target = frame._nodes[frame._nodes["go_id"] == term].index[0]
        if (source, target) in new_go_edges:
            new_go_edges[(source, target)].add("involved_in")
        else:
            new_go_edges[(source, target)] = {"involved_in"}

In [ ]:
frame.add_edges(new_go_edges.keys())
for e, types in new_go_edges.items():
    frame._edges["@type"][e] = types

## Import to Neo4j

In [ ]:
uri = "bolt://127.0.0.1:7687"
username = "neo4j"
password = "admin"

In [ ]:
for c in frame.node_properties():
    frame.node_prop_as_category(c)
frame.rename_node_properties({
    p: p.replace(" ", "_") for p in frame.node_properties()
})
frame._edges["@type"] = frame._edges["@type"].apply(
    lambda x: {el.replace(" ", "_") for el in x})
frame._edges["@type"] = frame._edges["@type"].apply(
    lambda x: {el.replace("NOT|enables", "NOT_enables") for el in x})
frame._edges["@type"] = frame._edges["@type"].apply(
    lambda x: {el.replace("NOT|involved_in", "NOT_involved_in") for el in x})
frame._edges["@type"] = frame._edges["@type"].apply(
    lambda x: {el.replace("NOT|located_in", "NOT_located_in") for el in x})

In [ ]:
view = pgframe_to_neo4j(
    frame, uri=uri, username=username, password=password,
    node_label="ONTOLOGY_CLASS",
    node_types_as_labels=True,
    edge_types_as_labels=True,
    batch_size=10)

In [ ]:
label_top_level(view.driver)
label_entities(view.driver)

In [ ]:
queries = [
    "match (m)-[:IS_SUBCLASS_OF|IS_INSTANCE_OF*..]->(n {id: 'cellular_component'}) SET m:GENE_ONTOLOGY_CLASS",
    "match (m)-[:IS_SUBCLASS_OF|IS_INSTANCE_OF*..]->(n {id: 'biological_process'}) SET m:GENE_ONTOLOGY_CLASS",
    "match (m)-[:IS_SUBCLASS_OF|IS_INSTANCE_OF*..]->(n {id: 'molecular_function'}) SET m:GENE_ONTOLOGY_CLASS",
    "match (m)-[:IS_SUBCLASS_OF|IS_INSTANCE_OF*..]->(n {id: 'Gene'}) SET m:GENE_ONTOLOGY_CLASS, n:GENE_ONTOLOGY_CLASS"
]
for q in queries:
    execute(view.driver, q)

In [ ]:
# Reconnect transitive orthologues
q = (
    """
    match (a)-[:hasOrthologue]-(b)-[:hasOrthologue]-(c)
    merge (a)-[:hasOrthologue]->(c)
    """
)
execute(view.driver, q)
q = (
    """
    match (n)-[:hasOrthologue]->(m)
    merge (m)-[:hasOrthologue]->(n)
    """
)
execute(view.driver, q)